In [1]:
from cellitaire.environment.agents.PPOAgent import Agent
from cellitaire.environment.cellitaire_env import CellitaireEnv
from cellitaire.environment.rewards.reward import *
from cellitaire.environment.rewards.foundation_rewards import *

board_rows = 7
board_cols = 12
num_reserved = 6
test_reward = CombinedReward([
    #PlacedCardInFoundationReward(weight=6),
    WinReward(weight=100, rows=board_rows, cols=board_cols),
    #ConstantReward(weight=0.5),
    ScalingPlacedCardInFoundationReward(weight=1, rows=board_rows, cols=board_cols),
    #PlayedLegalMoveReward(weight=1, rows=board_rows, cols=board_cols, num_reserved = num_reserved),
    #PeriodicPlacedCardInFoundationReward(weight=4, reward_period=3),
    #CreatedMovesReward(weight=1, num_reserved=num_reserved, foundation_count_dropoff=30)
])
N = 10000
batch_size = 2000
n_epochs = 5
alpha = 0.00003

pygame 2.6.1 (SDL 2.28.4, Python 3.13.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import time
import numpy as np

try:
    best_score = -1000
    score_history = []
    max_score = 0
    episodes_without_best = 0
    
    demo_env = CellitaireEnv(test_reward, rows=board_rows, cols=board_cols, num_reserved=num_reserved, max_moves=1200, max_illegal_moves=1200, render_mode='human', frame_rate=0.1)
    demo_env.render()
    
    demo_agent = Agent(n_actions=demo_env.action_space.n, 
        input_dims=(board_rows * board_cols * 4 + 6,), batch_size=batch_size, 
                    alpha=alpha, n_epochs=n_epochs, checkpoint_dir='tmp/testing_parallel')
    demo_agent.load_models()
    
    learn_iters = 0
    avg_score = 0
    n_steps = 0
    i = 0
    while True:
        demo_agent.load_models()
        demo_env.reset()
        observation = demo_env.get_state()
        done = False
        score = 0
        while not done:
            action, _, _ = demo_agent.choose_legal_action_mostly(observation, demo_env.get_legal_actions_as_int())
            observation_, reward, done, _, info = demo_env.step(action)
            n_steps += 1
            score += reward
            observation = observation_
        i += 1
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        
        max_score = max(max_score, score)
        
        if avg_score > best_score and i > N:
            best_score = avg_score
            recent_std = np.std(score_history[-100:])
            #agent.save_models()
            episodes_without_best = 0
            print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done} *')
        else:
            episodes_without_best += 1
            
        if episodes_without_best % 100 == 0 and episodes_without_best > 0:
            recent_std = np.std(score_history[-100:])
            print(f'episode {i:>5} | score {score:>6.1f} | avg {avg_score:>6.1f} | std {recent_std:>6.2f} | max score {max_score:>5.1f} | learning steps {learn_iters:>5} | done {done}')
        time.sleep(8)
        print(score)
except:
    demo_env.close()

FAILED TO LOAD
Error(s) in loading state_dict for CriticNetwork:
	size mismatch for critic.0.weight: copying a param with shape torch.Size([4096, 342]) from checkpoint, the shape in current model is torch.Size([2048, 342]).
	size mismatch for critic.0.bias: copying a param with shape torch.Size([4096]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for critic.1.weight: copying a param with shape torch.Size([4096]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for critic.1.bias: copying a param with shape torch.Size([4096]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for critic.3.weight: copying a param with shape torch.Size([4096, 4096]) from checkpoint, the shape in current model is torch.Size([2048, 2048]).
	size mismatch for critic.3.bias: copying a param with shape torch.Size([4096]) from checkpoint, the shape in current model is torch.Size([2048]).
	size mismatch for critic

In [4]:
demo_env.close()